*   **Año:** 2024
*   **Alumno/a:** Maria Guadalupe Echeverria
*   **Legajo:** 141821621

# Pydantic
Pydantic es una libreria rapida y extensible que nos permite validar datos usando tipos de datos de Python.


Primero, importe `pydantic`.

In [ ]:
import pydantic

Supongamos que tenemos una lista de clientes (llamemoslo en el codigo `Client`). Los clientes tienen dos campos: DNI (un entero) y nacionalidad (un `string`). Cree el modelo base de `Client` (en forma de clases de Python).

In [ ]:
from pydantic import BaseModel

class Client (BaseModel):
  DNI: int
  nacionalidad : str

Cree a un usuario con documento 39.755.010 y nacionalidad 'Argentina'. Muestre todos sus campos.

In [ ]:
class Usuario (BaseModel):
  DNI: int = '39.755.010'
  nacionalidad : str = 'Argentina'


Intente crear al usuario con un documento en forma de `string`. Deberia fallar...

In [ ]:
class Usuario (BaseModel):
  DNI: str = 39.755.010
  nacionalidad : str = 'Argentina'

#falla

SyntaxError: invalid syntax (<ipython-input-5-076bf012ba74>, line 2)

Hemos detectado que ciertos clientes tienen nacionalidades que no existen. Ademas, hay numeros de documento negativos y se tiene que poder agregar la fecha de registro de los clientes (que no pueden ser del futuro). Cambiar la definicion del cliente para que estas cosas no ocurran. Despues de la siguiente celda, cree otras 3 mas probando un caso donde deberia funcionar y otros dos en los que no.

In [ ]:
from pydantic import BaseModel, create_model, Field, validator
from typing import List
from datetime import date

nacionalidades_validas = ['Argentina', 'Chile', 'Brasil', 'Uruguay', 'Paraguay']
class Client(BaseModel):
    DNI: int = Field(..., gt=0, description="DNI debe ser un número positivo")
    nacionalidad: str
    fecha_registro: date

    @validator('nacionalidad')
    def validar_nacionalidad(cls, value):
        if value not in nacionalidades_validas:
            raise ValueError(f'Nacionalidad {value} no es válida.')
        return value

    @validator('fecha_registro')
    def validar_fecha_registro(cls, value):
        if value > date.today():
            print("error, fecha inválida:", value)
        return value


<ipython-input-7-86db358b31f0>:11: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  @validator('nacionalidad')
<ipython-input-7-86db358b31f0>:17: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  @validator('fecha_registro')


In [ ]:
# Caso que funciona
client1 = Client(DNI=12345678, nacionalidad='Argentina', fecha_registro=date(2023,5,1))
print(client1)


DNI=12345678 nacionalidad='Argentina' fecha_registro=datetime.date(2023, 5, 1)


In [ ]:
# Caso que debería fallar (DNI negativo)

client2 = Client(DNI=-12345678, nacionalidad="Argentina", fecha_registro=date(2023, 5, 1))
print(client2)


ValidationError: 1 validation error for Client
DNI
  Input should be greater than 0 [type=greater_than, input_value=-12345678, input_type=int]
    For further information visit https://errors.pydantic.dev/2.9/v/greater_than

In [ ]:
# Caso que debería fallar (fecha de registro en el futuro)

client3 = Client(DNI=87654321, nacionalidad="Uruguay", fecha_registro=date(2025, 1, 1))
print(client3)


error, fecha inválida: 2025-01-01
DNI=87654321 nacionalidad='Uruguay' fecha_registro=datetime.date(2025, 1, 1)


# Pandera
Hacer lo que se pidio en la ultima celda anterior, pero con Pandera.

In [ ]:
import pandera as pa
from datetime import date
import pandas as pd

# Lista de países válidos
PAISES_VALIDOS = ['Argentina', 'Brasil', 'Chile', 'Uruguay', 'Paraguay']

# Defino el esquema
cliente_schema = pa.DataFrameSchema({
    "DNI": pa.Column(int, checks=pa.Check.ge(1)),  # DNI debe ser mayor o igual a 1
    "nacionalidad": pa.Column(str, checks=pa.Check.isin(PAISES_VALIDOS)),  # Nacionalidad válida
    "fecha_registro": pa.Column(pa.DateTime, checks=pa.Check.le(date.today())),  # Fecha no puede ser futura
})

# Df de prueba
clientes = pd.DataFrame({
    "DNI": [12345678, -23456789, 87654321],
    "nacionalidad": ["Argentina", "Perú", "Brasil"],
    "fecha_registro": [date(2023, 5, 1), date(2025, 1, 1), date(2022, 12, 15)]
})

try:
    cliente_schema.validate(clientes)
    print("Datos válidos")
except pa.errors.SchemaError as e:
    print(f"Error en la validación: {e}")





Error en la validación: Column 'DNI' failed element-wise validator number 0: greater_than_or_equal_to(1) failure cases: -23456789
